# 12.d: The Method of Differences

> Order and simplification are the first steps toward the mastery of a subject.
>
> — Thomas Mann

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- Use the method of finite differences to determine the degree of a polynomial sequence.
- Explain the Principle of Degree Reduction.
- Use the method of differences to derive the coefficients of a quadratic function.
- Create a subplot visualization to show a function and its successive differences.

## 📚 Prerequisites

This notebook builds on concepts from the previous lessons. Before you begin, make sure you are comfortable with:
- Concepts from [Notebook 12.b: Finding Linear Patterns](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/12.b-finding-linear-patterns.ipynb), including the Method of First Differences.
- Concepts from [Notebook 12.c: Cracking the Quadratic Code](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/12.c-cracking-the-quadratic-code.ipynb), including the Method of Second Differences.

*Estimated Time: 50 minutes*

---

[Return to Table of Contents](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/table-of-contents.ipynb)

## Introduction: From Detective to Master Builder

In the last two notebooks, we learned to act like data detectives. We saw that first differences reveal linear patterns and second differences reveal quadratic ones. Our detective work has been successful, but it raises two big questions:

1.  **When do we stop?** How many times do we need to take the differences before we can be sure we've found the pattern?
2.  **Now what?** Once we've identified a pattern (like "quadratic"), how do we turn that knowledge into the actual formula that created the sequence?

In this notebook, we will answer both questions. We will formalize our detective work into a powerful, general tool called the **Method of Finite Differences**. This will give us a clear stopping condition and a reliable recipe for finding the function behind a sequence, turning us from pattern-finders into function-builders.

## 🛠️ Setup: Our Detective Kit

We only need one tool from our kit for this notebook: our trusted `calculate_differences` function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_differences(sequence):
    if not sequence or len(sequence) < 2:
        return [np.nan] * len(sequence)

    differences = [np.nan]
    for i in range(1, len(sequence)):
        # Check if either value is nan before appending
        if np.isnan(sequence[i]) or np.isnan(sequence[i-1]):
            differences.append(np.nan)
        else:
            differences.append(sequence[i] - sequence[i-1])
    return differences

## The "Why": When Do We Stop Taking Differences?

A key question in this process is, "How do I know when I'm done?" Let's revisit our thrown ball data from the last notebook and take the differences until we see a clear pattern emerge.

In [ ]:
# The height of the ball thrown from the bridge at t = 0, 1, 2, 3, 4, 5
heights = [10, 55, 90, 115, 130, 135]

# Calculate the first, second, and third differences
d1 = calculate_differences(heights)
d2 = calculate_differences(d1)
d3 = calculate_differences(d2)

print(f"Original Sequence (h): {heights}")
print(f"First Differences (d1):   {d1}")
print(f"Second Differences (d2):  {d2}")
print(f"Third Differences (d3):   {d3}")

This is the key! Notice that the **second difference** is constant, and the **third difference** is all zeros (ignoring the `nan` placeholders). This isn't a coincidence; it's the secret to knowing when to stop.

> **The Method of Differences: Stopping Rule**
> Your **goal** is to find the first difference sequence that is **constant and not zero**.
> You **confirm** you are done because the *very next* difference sequence will be **all zeros**.
> (And if you keep going, you'll just get more and more zeros!)

### The Principle of Degree Reduction

Why does this rule work? It's because of a powerful underlying principle: **each time you take a difference, you reduce the degree of the underlying polynomial by one.**

| Sequence Type | Degree | 1st Difference is... | 2nd Difference is... | 3rd Difference is... |
| :--- | :---: | :--- | :--- | :--- |
| **Quadratic** | 2 | Linear (deg 1) | Constant (deg 0) | Zero |
| **Linear** | 1 | Constant (deg 0) | Zero | |
| **Constant** | 0 | Zero | | |

This is why the method is a systematic process of simplification. We keep taking differences until we reach a simple, constant value, and we know the next step will be zero.

### 🐍 New Python Tool: Subplot Visualization

A powerful way to see this degree reduction in action is to plot a sequence and its successive differences all at once. We can do this using `matplotlib`'s `subplots` feature, which lets us create a figure with multiple plots stacked vertically.

In [ ]:
def plot_differences(sequence):
     """Calculates and plots a sequence and its first two differences."""
     d1 = calculate_differences(sequence)
     d2 = calculate_differences(d1)
     n = range(len(sequence))
     fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
     fig.suptitle('Visualizing the Method of Differences', fontsize=16)
     # Plot Original Sequence
     ax1.plot(n, sequence, 'o-')
     ax1.set_title("Original Sequence (Height)")
     ax1.set_ylabel("Height (m)")
     ax1.grid(True)

     # Plot First Difference
     ax2.plot(n, d1, 'o-', color='r')
     ax2.set_title("First Difference (Avg. Velocity)")
     ax2.set_ylabel("Velocity (m/s)")
     ax2.grid(True)

     # Plot Second Difference
     ax3.plot(n, d2, 'o-', color='g')
     ax3.set_title("Second Difference (Avg. Acceleration)")
     ax3.set_ylabel("Accel. (m/s²)")
     ax3.set_xlabel("Time (s)")
     ax3.grid(True)

     plt.show()

# Let's visualize our ball's height data
plot_differences(heights)

## The "How": Finding Coefficients with the Leading Diagonal

The visualization makes it clear: if the second difference is a constant value, the original function was a quadratic. But how do we find the formula $h(t) = at^2 + bt + c$?

The secret lies in the first valid number of each difference sequence, which is called the **leading diagonal** of the difference table. Let's look at the general case for a quadratic function, where `n` starts at 0:

### What are `a`, `b`, and `c`?

This is a great question. In the last few notebooks, we proved that if the second difference is constant, the sequence must be **quadratic**.

In algebra, any quadratic function can be written in a **general form**:
$$T(n) = an^2 + bn + c$$

Think of this as a blueprint for all quadratic functions. The letters `a`, `b`, and `c` are called **coefficients**. They are the specific numbers that make one quadratic function different from another.

- `c` is the **constant term** (or the y-intercept).
- `b` is the coefficient of the **linear term** (the `n` part).
- `a` is the coefficient of the **quadratic term** (the `n^2` part).

Our job as data detectives is to figure out the exact values of `a`, `b`, and `c` for our specific sequence. We aren't pulling them out of thin air; we are starting with a blueprint that we *know* is correct (because the second difference was constant) and now we are just solving for the missing numbers in that blueprint.

<details>
<summary>Optional: Click here to see the step-by-step algebra for deriving the recipe.</summary>

### Deriving the Recipe

How do we get the recipe for the coefficients? Let's build it step-by-step using algebra.

**1. The `c` coefficient**
The formula is $T(n) = an^2 + bn + c$. If we look at the very first term where `n=0`:
$T(0) = a(0)^2 + b(0) + c = 0 + 0 + c = c$.
So, the first term of our sequence, `T[0]`, is always equal to `c`!

**2. The `a` and `b` coefficients**
Now let's look at the first difference, $d_1(1) = T(1) - T(0)$.
$T(1) = a(1)^2 + b(1) + c = a+b+c$.
So, $d_1(1) = (a+b+c) - c = a+b$.
This gives us our second equation: $a+b = d_1[1]$.

**3. The `a` coefficient**
Finally, let's look at the second difference, $d_2(2) = d_1(2) - d_1(1)$.
We already know $d_1(1) = a+b$. We need $d_1(2) = T(2) - T(1)$.
$T(2) = a(2)^2 + b(2) + c = 4a+2b+c$.
$d_1(2) = (4a+2b+c) - (a+b+c) = 3a+b$.
Now we can find the second difference:
$d_2(2) = (3a+b) - (a+b) = 2a$.
This gives our final key equation: $2a = d_2[2]$.

This step-by-step derivation leads us directly to the recipe, which is summarized in the table below.

| n | $T(n) = an^2+bn+c$ | $d_1(n)$ | $d_2(n)$ |
|:-:|:---|:---|:---|
| 0 | $T(0)=a \cdot 0^2 + b \cdot 0 + c$<br> $=c$ | | |
| 1 | $T(1)=a \cdot 1^2 + b \cdot 1 + c$<br>$=a+b+c$ | $d_1(1)=T(1) - T(0)$<br>$=(a+b+c) - c$<br>$=a + b$| |
| 2 | $T(2)=a \cdot 2^2 + b \cdot 2 + c$<br>$=4a+2b+c$ | $d_1(2)=T(2) - T(1)$<br>$=(4a+2b+c) - (a+b+c)$<br>$=3a+b$| $d_2(2)=d_1(2) - d_1(1)$<br>$=(3a+b)-(a+b)$<br>$=2a$|

By looking at the first usable values from the table (the leading diagonal), we get a **system of equations**. This is a set of clues (equations) that must all be true at the same time. Our three clues are:

1. $2a = d_2[2]$ (the first constant value of the 2nd difference)
2. $a+b = d_1[1]$ (the first calculated value of the 1st difference)
3. $c = T[0]$ (the first term of the original sequence)

We can solve these for $a$, $b$, and $c$ in that order!
</details>

### A Recipe for Finding Coefficients

Here is our step-by-step recipe for a quadratic sequence:

1.  **Find `a`:** $a = \frac{d_2[2]}{2}$
2.  **Find `b`:** $b = d_1[1] - a$
3.  **Find `c`:** $c = T[0]$

In [ ]:
# 1. Find a
a = d2[2] / 2

# 2. Find b
b = d1[1] - a

# 3. Find c
c = heights[0]

print(f"The coefficients are: a = {a}, b = {b}, c = {c}")

### Verifying the Formula

Our recipe gives us $a=-5.0$, $b=50.0$, and $c=10.0$. This means the formula should be $h(t) = -5t^2 + 50t + 10$. Let's test it!

In [ ]:
def found_formula(n):
    # Our derived coefficients
    a = -5.0
    b = 50.0
    c = 10.0
    return a * n**2 + b * n + c

# Generate the sequence from our new formula
generated_sequence = []
for i in range(len(heights)):
    generated_sequence.append(found_formula(i))

print(f"Original sequence:  {heights}")
print(f"Generated sequence: {generated_sequence}")

# Check if they match
if heights == generated_sequence:
    print('Success! The formula is correct.')
else:
    print('Something went wrong...')

### 🎯 Optional Challenge: The Fudge Pricing Puzzle

Want some more practice? Here is a different scenario. A candy shop has a 3-part pricing model for its specialty square fudge pieces:
1.  A flat fee for the bag.
2.  A price for the box, which is proportional to the side length `L` of the fudge.
3.  A price for the fudge itself, which is proportional to the area (`L^2`).

This results in a quadratic pricing formula: `Price(L) = a*L^2 + b*L + c`.

Here are the prices:
- Just a bag (0x0 fudge): $2.00
- 1x1 inch square: $4.00
- 2x2 inch square: $7.00
- 3x3 inch square: $11.00
- 4x4 inch square: $16.00

Your task is to find the complete formula by determining `a`, `b`, and `c`, and then use it to price a new **5x5 inch** square.

<details>
<summary>Hint: How do I start?</summary>

Create a list of the prices, starting with the `L=0` case for the bag. Then, use the `calculate_differences` function twice to get `d1` and `d2`.

</details>
<details>
<summary>Hint: Applying the recipe</summary>

Once you have the original sequence `T`, the first difference `d1`, and the second difference `d2`, you can use the first valid number from each list to find the coefficients:
- `c = T[0]`
- `b = d1[1] - a`
- `a = d2[2] / 2`

Remember to solve for `a` first!

</details>

In [ ]:
# Prices for L = 0, 1, 2, 3, 4
fudge_prices = [2.00, 4.00, 7.00, 11.00, 16.00]

# 1. Calculate the first and second differences
d1 = [] # YOUR CODE HERE
d2 = [] # YOUR CODE HERE
print(f"d1: {d1}")
print(f"d2: {d2}")
print("---S")

# 2. Use the recipe to find a, b, and c
a = 0 # YOUR CODE HERE
b = 0 # YOUR CODE HERE
c = 0 # YOUR CODE HERE
print(f"Coefficients: a={a}, b={b}, c={c}")
print("---S")

# 3. Define the formula and find the price for a 5x5 square
def price_formula(L):
    # YOUR CODE HERE
    return 0

price_5x5 = price_formula(5)
print(f"The price for a 5x5 inch square of fudge is: ${price_5x5:.2f}")

<details>
<summary>Click to see a possible solution</summary>

```python
# Prices for L = 0, 1, 2, 3, 4
fudge_prices = [2.00, 4.00, 7.00, 11.00, 16.00]

# 1. Calculate the first and second differences
d1 = calculate_differences(fudge_prices)
d2 = calculate_differences(d1)
print(f"d1: {d1}")
# Expected output: d1: [nan, 2.0, 3.0, 4.0, 5.0]
print(f"d2: {d2}")
# Expected output: d2: [nan, nan, 1.0, 1.0, 1.0]
print("---S")

# 2. Use the recipe to find a, b, and c
c = fudge_prices[0]
a = d2[2] / 2
b = d1[1] - a
print(f"Coefficients: a={a}, b={b}, c={c}")
# Expected output: Coefficients: a=0.5, b=1.5, c=2.0
print("---S")

# 3. Define the formula and find the price for a 5x5 square
def price_formula(L):
    return a * L**2 + b * L + c

price_5x5 = price_formula(5)
print(f"The price for a 5x5 inch square of fudge is: ${price_5x5:.2f}")
# Expected output: The price for a 5x5 inch square of fudge is: $22.00
```

</details>

## 🎉 Well Done!

You have now mastered the Method of Finite Differences for quadratic sequences! You have moved from being a data detective, able to spot a pattern, to a master builder, able to reconstruct the formula that creates the pattern. This is a powerful step in computational problem-solving.

### Key Takeaways
- The **(k+1)-th difference** of a polynomial of degree *k* is always zero.
- This gives us a clear **stopping condition** for our detective work.
- The **leading diagonal** of the difference table gives us the keys to solve for the coefficients $a$, $b$, and $c$ in a systematic way.

### Next Up: A Glimpse of Calculus 🚀

This process of taking differences might feel familiar if you've seen calculus before. In our next and final notebook of this series, [Notebook 12.e: A Glimpse of Calculus](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/12.e-a-glimpse-of-calculus.ipynb), we will explore the fascinating connection between the Method of Differences and the fundamental concepts of calculus.

---
[Return to Table of Contents](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/table-of-contents.ipynb)